In [1]:
import numpy as np
import os
from hdf5storage import loadmat, savemat
from multitools import gamma_GC, make_pos_def
from eda import KnapsackEDA, generate_example_data
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from itertools import islice, combinations
from numpy import random
from scipy.spatial.distance import jensenshannon

In [ ]:
#parameters
n_items = 60
n_selected = 6
n_obj = 3
n_con = 1
shape = [3.0, 4.0, 2.0, 8.0]
scale = [2.0, 3, 2, 1.0]
r = np.array([      
    [1.0, -0.4, -0.5, 0.3],
    [-0.4, 1.0, 0.5, 0.4],
    [-0.5, 0.5, 1.0, 0.2],
    [0.3, 0.4, 0.2, 1.0],
])
capacity = int(shape[-1]*scale[-1]*n_selected)
pop_size = 1000
generations = 100 # do not matter if check convergence
max_no_improve_gen = 10


In [ ]:
for run in range(5):
    items_seed = 111+run
    eda_seed = 1123+run
    # Generate data
    items = generate_example_data(r, shape, scale, n_items=n_items, seed=items_seed)


    # Run EDA
    eda = KnapsackEDA(
        items=items,
        capacity=capacity,
        n_selected=n_selected,
        n_obj=n_obj,
        pop_size=pop_size,
        generations=generations,
        max_no_improve_gen=max_no_improve_gen,
        seed=eda_seed
    )
    #organize results    
    results = eda.run()
    js_div_list = results['js_div_list']
    distribution_table = results['distribution_table']
    pareto_indices_table = []
    pareto_front_table = []
    for j in range(len(results['pareto_front_table'])):
        pareto_indices_table.append(np.unique(np.sort(results['pareto_indices_table'][j],axis = 1),axis =0))
        pareto_front_table.append(np.unique(results['pareto_front_table'][j], axis=0))
    # Save results
    sim = dict()
    sim['n_items'] = n_items
    sim['n_selected'] = n_selected
    sim['n_obj'] = n_obj
    sim['n_con'] = n_con
    sim['shape'] = shape
    sim['scale'] = scale
    sim['r'] = r
    sim['capacity'] = capacity
    sim['pop_size'] = pop_size
    sim['max_no_improve_gen'] = max_no_improve_gen
    sim['items'] = items
    sim['items_seed'] = items_seed
    sim['eda_seed'] = eda_seed
    sim['js_div_list'] = js_div_list
    sim['distribution_table'] = distribution_table
    sim['pareto_indices_table'] = pareto_indices_table
    sim['pareto_front_table'] = pareto_front_table      
    #save to file
    if not os.path.exists('./results/'):
        os.makedirs('./results/')
    output_dir = './results/'
    savemat(os.path.join(output_dir, f"knapsack_eda_sim_{n_items}_{n_selected}_{run}.mat"), sim, store_python_metadata=True)

Generation 1 (no improve count: 0)
Generation 2 (no improve count: 0)
Generation 3 (no improve count: 0)
Generation 4 (no improve count: 0)
Generation 5 (no improve count: 0)
Generation 6 (no improve count: 0)
Generation 7 (no improve count: 0)
Generation 8 (no improve count: 0)
Generation 9 (no improve count: 0)
Generation 10 (no improve count: 1)
Generation 11 (no improve count: 0)
Generation 12 (no improve count: 0)
Generation 13 (no improve count: 0)
Generation 14 (no improve count: 0)
Generation 15 (no improve count: 1)
Generation 16 (no improve count: 0)
Generation 17 (no improve count: 0)
Generation 18 (no improve count: 1)
Generation 19 (no improve count: 0)
Generation 20 (no improve count: 0)
Generation 21 (no improve count: 0)
Generation 22 (no improve count: 0)
Generation 23 (no improve count: 1)
Generation 24 (no improve count: 0)
Generation 25 (no improve count: 0)
Generation 26 (no improve count: 1)
Generation 27 (no improve count: 2)
Generation 28 (no improve count: 3)
G

In [4]:
# output_dir = "./results"
# if len(distribution_table) > 0:
#     dist_df = pd.DataFrame(np.vstack(distribution_table))
#     dist_df.index.name = 'generation'
#     dist_df.columns = [f"item_{i}" for i in range(dist_df.shape[1])]
#     dist_df.to_csv(os.path.join(output_dir, f"distribution_table_{n_items}_{n_selected}_{run}.csv"))

# if len(js_div_list) > 0:
#     js_df = pd.DataFrame({
#         'generation': np.arange(1, len(js_div_list) + 1, dtype=int),
#         'js_divergence': js_div_list
#     })
#     js_df.to_csv(os.path.join(output_dir, f"js_div_list_{n_items}_{n_selected}_{run}.csv"), index=False)

# if len(pareto_indices_table) > 0:
#     np.savez_compressed(
#         os.path.join(output_dir, f"pareto_indices_table_{n_items}_{n_selected}_{run}.npz"),
#         **{f"gen_{i+1}": arr for i, arr in enumerate(pareto_indices_table)}
#     )
# if len(pareto_front_table) > 0:
#     np.savez_compressed(
#         os.path.join(output_dir, f"pareto_front_table_{n_items}_{n_selected}_{run}.npz"),
#         **{f"gen_{i+1}": arr for i, arr in enumerate(pareto_front_table)}
#     )

In [5]:
# sim = dict()
# sim['n_items'] = n_items
# sim['n_selected'] = n_selected
# sim['n_obj'] = n_obj
# sim['n_con'] = n_con
# sim['shape'] = shape
# sim['scale'] = scale
# sim['r'] = r
# sim['capacity'] = capacity
# sim['pop_size'] = pop_size
# sim['max_no_improve_gen'] = max_no_improve_gen
# sim['items'] = items
# sim['items_seed'] = items_seed
# sim['eda_seed'] = eda_seed
# sim['js_div_list'] = js_div_list
# sim['distribution_table'] = distribution_table
# sim['pareto_indices_table'] = pareto_indices_table
# sim['pareto_front_table'] = pareto_front_table      
# savemat(os.path.join(output_dir, f"knapsack_eda_sim_{n_items}_{n_selected}_{run}.mat"), sim, store_python_metadata=True)